In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Lambda, Bidirectional, LSTM, Dense
from keras_bert import load_trained_model_from_checkpoint
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from datetime import datetime
from collections import Counter
import keras.callbacks
import re
import codecs
import time

Using TensorFlow backend.


In [3]:
import os
import codecs
import pandas as pd
import numpy as np

bert_dir = r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\bert'
config_path = os.path.join(bert_dir, 'bert_config.json')
checkpoint_path = os.path.join(bert_dir, 'bert_model.ckpt')
dict_path = os.path.join(bert_dir, 'vocab.txt')

# 0. 載入資料

In [4]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data = data[data['aml_label'] == 1]

In [5]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data['name'] = data['name'].apply(lambda x: eval(x))
data['content'] = data['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data['content'] = data['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data['content'] = data['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
data['content'] = data['content'].apply(lambda x: re.sub('「.{1,4}」', '', x))
data['content'] = data['content'].apply(lambda x: re.sub('｜.', '。', x))

In [6]:
train = data

In [ ]:
#data2 = pd.read_csv(r'C:\Users\rocker\extra data\正式賽1~4.csv', encoding='cp950')
#data2['name'] = data2['name'].apply(lambda x: eval(x))
#data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
#data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
#                                                   .replace('報導', '＞')
#                                                   .replace('▲', '')\
#                                                   .replace('。　', '。')\
#                                                   .replace('', '')\
#                                                   .replace('.', '')\
#                                                   .replace(' ', '')\
#                                                   .replace('“', '「')\
#                                                   .replace('”', '」'))
#data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
#data2['content'] = data2['content'].apply(lambda x: re.sub('「.{1,4}」', '', x))
#data2['content'] = data2['content'].apply(lambda x: re.sub('｜.', '。', x))
#data2 = data2[['news_id', 'name', 'content', 'aml_label']]
#train2 = data2
#train = train.append(train2)

In [7]:
data2 = pd.read_csv(r'C:\Users\rocker\extra data\test.csv')
data2 = data2.drop(['title', 'link'], axis=1)
data2['people_list'] = data2['people_list'].apply(lambda x: eval(x))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
data2['content'] = data2['content'].apply(lambda x: re.sub('「.{1,4}」', '', x))
data2.columns = ['news_id', 'content', 'aml_label', 'name']
data2 = data2[['news_id', 'name', 'content', 'aml_label']]
train2 = data2

In [8]:
data2 = pd.read_csv(r'C:\Users\rocker\extra data\新增資料夾\ltn_檢察官_法官_0719_過模型.csv', encoding='cp950')
data2 = data2.drop(['title', 'link', 'people_list'], axis=1)
data2['aml_label'] = data2['Name_prediction'].apply(lambda x: 0 if x == '[]' else 1)
data2['news_id'] = range(1, len(data2)+1)
data2['Name_prediction'] = data2['Name_prediction'].apply(lambda x: eval(x))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
data2['content'] = data2['content'].apply(lambda x: re.sub('「.{1,4}」', '', x))
train3 = data2
train3.columns = ['news_id', 'content', 'name', 'aml_label']
train3 = train3[['news_id', 'name', 'content', 'aml_label']]

# 1. 建立犯罪模型

In [9]:
def create_tokenizer(dict_path):
    
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
            
    return token_dict

def transfer(i):
    
    if i != 0:
        return 1
    else:
        return 0

def encoded(tokenizer, data, maxlen):
    
    x, y, z = [], [], []
    if 'content' in data.columns:
        for content in data['content']:
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = list(map(lambda x: 1 if x != 0 else 0, x1))
            x.append(x1)
            y.append(x2)
            z.append(x3)
    elif 'Sentence' in data.columns:
        for content in data['Sentence']:
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = list(map(lambda x: 1 if x != 0 else 0, x1))
            x.append(x1)
            y.append(x2)
            z.append(x3)
            
    return x, y, z


def rebuild_sentence(content, maxlen):
    
    if len(content) > maxlen:
        return content[:round(maxlen/2)-1]+ '。' + content[len(content) - (maxlen - round(maxlen/2))+2:]
    else:
        return content


def encoded_crime(tokenizer, data, maxlen):
    
    x, y, z = [], [], []
    if 'content' in data.columns:
        for content in data['content']:

            content = rebuild_sentence(content, maxlen)
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = list(map(lambda x: 1 if x != 0 else 0, x1))
            x.append(x1)
            y.append(x2)
            z.append(x3)
    elif 'Sentence' in data.columns:
        for content in data['Sentence']:
            
            content = rebuild_sentence(content, maxlen)    
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = list(map(lambda x: 1 if x != 0 else 0, x1))
            x.append(x1)
            y.append(x2)
            z.append(x3)
            
    return x, y, z

In [10]:
token_dict = create_tokenizer(dict_path)
tokenizer = Tokenizer(token_dict)

In [11]:
maxlen = 512
batch_size = 8
epochs = 3

In [12]:
def bert_LSTM_model():
    
    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen)
    sequence_output = model.layers[-9].output
    sequence_output = Bidirectional(LSTM(128, return_sequences=False))(sequence_output)
    output = Dense(1, activation='sigmoid')(sequence_output)
    model = Model(model.input, output)
    
    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    return model

model = bert_LSTM_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
train1_1 = train[train.aml_label == 1]
train0 = train[(~train.content.str.contains('檢察官|法官|殺人|傷人|酒駕|性侵|潑漆|大麻|逃逸|罷免|債|過失|警|毒')) & (train.aml_label == 0) & (~train.content.str.contains('罪')) & (~train.content.str.contains('詐欺|內線交易|背信|簽賭'))]
crime_train = train1_1.append(train0)
crime_train['news_id'] = range(1, len(crime_train)+1)

In [14]:
label = np.asarray(crime_train['aml_label'])
input_id, segment_id, mask_input = encoded_crime(tokenizer, crime_train, maxlen)

In [15]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [16]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )


Epoch 1/3
2999/2999 [==============================] - 287s 96ms/step - loss: 0.0764 - acc: 0.9677
Epoch 2/3
2999/2999 [==============================] - 286s 95ms/step - loss: 0.0085 - acc: 0.9963
Epoch 3/3
2999/2999 [==============================] - 288s 96ms/step - loss: 0.0016 - acc: 1.0000


# 2. 建立 AML 犯罪模型

In [18]:
maxlen_aml = 512
batch_size = 8
epochs = 4

In [19]:
aml_train1_1 = train[train.aml_label == 1]
aml_train1_2 = train2[train2.aml_label == 1]
aml_train1_3 = train3[train3.aml_label == 1]

aml_train2_1 = train[(train.content.str.contains('檢察官|法官|殺人|傷害|傷人|酒駕|性侵|潑漆|大麻|逃逸|罷免|債|過失|警|毒')) & (train.aml_label == 0) & (train.content.str.contains('罪')) & (~train.content.str.contains('洗錢|偽造|走私|貪污|侵占|販毒|賄|詐|內線交易|背信|香港|簽賭'))]
aml_train2_2 = train2[(train2.content.str.contains('檢察官|法官|殺人|傷害|傷人|酒駕|性侵|潑漆|大麻|逃逸|罷免|債|過失|警|毒')) & (train2.aml_label == 0) & (train2.content.str.contains('罪')) & (~train2.content.str.contains('洗錢|偽造|走私|貪污|侵占|販毒|賄|詐|內線交易|背信|香港|簽賭'))]
aml_train2_3 = train3[(train3.content.str.contains('檢察官|法官|殺人|傷害|傷人|酒駕|性侵|潑漆|大麻|逃逸|罷免|債|過失|警|毒')) & (train3.aml_label == 0) & (train3.content.str.contains('罪')) & (~train3.content.str.contains('洗錢|偽造|走私|貪污|侵占|販毒|賄|詐|內線交易|背信|香港|簽賭'))]

aml_train3 = pd.read_csv(r'C:\Users\rocker\extra data\model1.5 data.csv',encoding='cp950')
aml_train3 = aml_train3.drop(['title', 'link'], axis=1)
aml_train3['name'] = '[]'
aml_train3['name'] = aml_train3['name'].apply(lambda x: eval(x))

aml_train = aml_train1_1.append(aml_train1_2).append(aml_train1_3).append(aml_train2_1).append(aml_train2_2).append(aml_train2_3).append(aml_train3)
aml_train['news_id'] = range(1, len(aml_train)+1)

In [20]:
label = np.asarray(aml_train['aml_label'])
input_id, segment_id, mask_input = encoded_crime(tokenizer, aml_train, maxlen_aml)

In [21]:
def model_aml():
    
    model_aml = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_aml)
    sequence_output = model_aml.layers[-6].output
    sequence_output = Dense(64, activation='relu')(sequence_output) 
    output = Dense(1, activation='sigmoid')(sequence_output)
    model_aml = Model(model_aml.input, output)
    return model_aml

model_aml = model_aml()

In [22]:
set_trainable = False
for layer in model_aml.layers:
    if layer.name == 'Encoder-12-MultiHeadSelfAttention':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [23]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [24]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [25]:
model_aml.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model_aml.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )

Epoch 1/4
1546/1546 [==============================] - 49s 31ms/step - loss: 0.2960 - acc: 0.8803
Epoch 2/4
1546/1546 [==============================] - 48s 31ms/step - loss: 0.2021 - acc: 0.9263
Epoch 3/4
1546/1546 [==============================] - 47s 30ms/step - loss: 0.1345 - acc: 0.9450
Epoch 4/4
1546/1546 [==============================] - 47s 31ms/step - loss: 0.1046 - acc: 0.9664


# 3. 建立 NER 模型

In [26]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data['name'] = data['name'].apply(lambda x: eval(x))

In [27]:
data['content'] = data['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data['content'] = data['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data['content'] = data['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))

In [28]:
train = data

In [29]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content}, index=[66]), ignore_index=True)
    
    return data_more_split

In [30]:
maxlen_ner = 512
batch_size = 8
epochs = 3
input_shape = (maxlen_ner, )

In [31]:
#分句
train_ner = train.drop(['name', 'aml_label'], axis=1)
data_less = train_ner[train_ner['content'].str.len() <= 512]
data_more = train_ner[train_ner['content'].str.len() > 512]
data_more_split = split_content(data_more)
train_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

###  3-1 用CKIP做NER

In [ ]:
# 取得 NER input array (陳水扁貪汙 -> CKIP(陳水扁) -> 1 2 2 0 0)
def transfer_NER(data, tokenizer, maxlen):
    people_list = []
    first_name = ['申', '龔', '馮', '昌', '劉', '習', '陽', '顧', '鍾', '胡', '許', '魏', '傅', '季', '扶', '柳', '狄', '焦', '封', '李', '羿', '刁', '和', '邴',    '陸', '王', '杜', '能', '侯', '伍', '平', '竺', '樂', '繆', '欒', '湛',    '道', '花', '賴', '浦', '萬', '章', '宮', '勾', '邵', '印', '夏', '杭',    '溥', '左', '池', '公', '閻', '符', '奚', '臧', '羅', '空', '璩', '巴',    '酈', '范', '談', '金', '顏', '慎', '郭', '僪', '聞', '車', '闞', '相',    '童', '雙', '方', '莊', '容', '姚', '田', '薛', '閔', '翟', '簡',    '蔚', '茹', '淩', '戴', '余', '鞏', '房', '富', '牛', '饒', '計', '居',    '後', '舒', '席', '翁', '祝', '鬱', '訾', '隆', '匡', '弘', '曆', '範',    '越', '趙', '卻', '岑', '隗', '冷', '張', '山', '松', '柯', '嵇', '韓',    '蕭', '褚', '殳', '滕', '滿', '洪', '荀', '庾', '廖', '盧', '危', '竇',    '曾', '郎', '遊', '穀', '慕', '禹', '凌', '廉', '邢', '梁', '葉',    '郝', '終', '齊', '藺', '曹', '全', '高', '樊', '史', '桂', '廣', '段',    '江', '餘', '袁', '弓', '牧', '魚', '儲', '尚', '逄', '尹', '通', '懷',    '皮', '何', '倪', '包', '晁', '涂', '蓬', '屠', '巫', '須', '巢', '卞',    '楊', '成', '孟', '楚', '呂', '古', '毋', '伊', '賁', '喻', '糜',    '蔔', '艾', '藍', '龐', '諸', '別', '任', '管', '冀', '壽', '惠', '梅',    '孫', '從', '康', '常', '駱', '鞠', '沈', '黨', '沙', '鳳', '郁', '邊',    '仰', '溫', '路', '逮', '賀', '雷', '鈄', '明', '裴', '滑', '毛', '費',    '關', '時', '步', '麴', '裘', '蒲', '司', '查', '錢', '盛', '霍', '鮑',    '彭', '龍', '沃', '單', '勞', '秋', '祖', '殷', '茅', '敖', '郗', '石',    '鐘', '嚴', '畢', '燕', '姜', '經', '程', '厙', '柏', '汪', '婁', '胥',    '聶', '邰', '桑', '辛', '扈', '穆', '仲', '紅', '項', '師', '桓', '黃',    '堵', '貢', '詹', '朱', '蔡', '戈', '于', '甄', '束', '屈', '索', '晏',    '阮', '魯', '虞', '歐', '濮', '俞', '黎', '文', '應', '姬', '貝', '籍',    '莘', '戚', '鄭', '郜', '景', '宋', '宗', '昝', '卓', '蒯', '馬', '顔',    '蘇', '衛', '東', '瞿', '蒼', '莫', '邱', '潘', '家', '林', '芮', '麻',    '元', '武', '強', '鈕', '陳', '井', '於', '游', '耿', '柴', '荊', '韶',    '易', '宿', '施', '鹹', '秦', '班', '甯', '汲', '酆', '暴', '尤',    '祿', '苗', '權', '仇', '都', '羊', '榮', '陶', '支', '賈', '白', '葛',    '暨', '解', '靳', '伏', '唐', '華', '吉', '融', '豐', '安', '衡', '那',    '闕', '俄', '盍', '鄔', '蒙', '利', '鄂', '謝', '宓', '湯', '喬', '孔',    '養', '紀', '幹', '牟', '連', '宰', '蔣', '雍', '益', '寇', '祁', '熊',    '崔', '丁', '薊', '譚', '吳', '烏', '周', '農', '徐', '充', '向', '宦',    '董', '甘', '冉', '韋', '米', '鄒', '鄧', '戎', '水']
    label = np.zeros([len(data), maxlen])
    
    # 用CKIP抓出每個新聞的名字
    for index, (_, row) in enumerate(data.iterrows()):
        
        if index % 100 == 0:
            print(index)
        
        token = tokenizer.tokenize(row['content'][0:maxlen])

        if len(token) > maxlen:
            token = token[0:maxlen-1]
            token.append('[SEP]')

        y = np.zeros([maxlen])
        content = ''.join(token)
        
        #CKIP
        word_sentence_list = ws([content],
                    sentence_segmentation=True,
                    segment_delimiter_set={'?', '？', '!', '！', '。', ',', '，', ';', ':', '、'})
        pos_sentence_list = pos(word_sentence_list)
        entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
        
        people = [people for people in list(entity_sentence_list[0]) if (people[2] == 'PERSON') & (people[1] < maxlen)]
        people = [people for people in people if (len(people[3]) < 5) | ((len(people[3]) >= 5) & ('#' not in people[3]))]
        people.sort()
        people_list.append(people)
        
        #轉換成input array
        j = 0
        for person in people: 
            for i, _ in enumerate(token):        
                if token[i:i+len(person[3])] == list(person[3]):
                    if len(person) == 1:
                        y[i+j] = 1
                        token = token[i+1:]
                        j = i+j+1
                        break

                    y[i+j] = 1
                    y[i+j+1:i+j+len(person[3])] = 2
                    token = token[i+len(person[3]):]
                    j = i+j+len(person[3])
                    break
        label[index, :] = y
    
    #用不到 people_list，只是檢查用
    return people_list, label.reshape([label.shape[0], label.shape[1], 1])

In [ ]:
people_list, label = transfer_NER(train_ner, tokenizer, maxlen=maxlen_ner)

In [ ]:
people_list = pd.to_csv(r'C:\Users\rocker\ner.csv',encoding='utf_8_sig', index=False)

### 3-2 用人工辨識做NER

In [32]:
# 取得 NER input array (陳水扁貪汙 -> CKIP(陳水扁) -> 1 2 2 0 0)
def transfer_NER_array(data, people_list, tokenizer, maxlen):
    
    label = np.zeros([len(data), maxlen])
    
    # 用CKIP抓出每個新聞的名字
    for (index, (_, row)), people in zip(enumerate(data.iterrows()), people_list):
        
        if index % 100 == 0:
            print(index)
        
        token = tokenizer.tokenize(row['content'][0:maxlen])

        if len(token) > maxlen:
            token = token[0:maxlen-1]
            token.append('[SEP]')

        y = np.zeros([maxlen])
        content = ''.join(token)
        
        #轉換成input array
        j = 0
        for person in people: 
            for i, _ in enumerate(token):        
                if token[i:i+len(person[3])] == list(person[3]):
                    if len(person) == 1:
                        y[i+j] = 1
                        token = token[i+1:]
                        j = i+j+1
                        break

                    y[i+j] = 1
                    y[i+j+1:i+j+len(person[3])] = 2
                    token = token[i+len(person[3]):]
                    j = i+j+len(person[3])
                    break
        label[index, :] = y
    
    #用不到 people_list，只是檢查用
    return label.reshape([label.shape[0], label.shape[1], 1])

In [33]:
people = pd.read_csv(r'C:\Users\rocker\ner.csv',encoding='big5hkscs')

In [34]:
people_list = list(people['people_list'].apply(lambda x: eval(x)))

### 3-3 訓練模型

In [35]:
label = transfer_NER_array(train_ner, people_list, tokenizer, maxlen=maxlen_ner)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200


In [36]:
input_id, segment_id, mask_input = encoded(tokenizer, train_ner, maxlen=maxlen_ner)

In [37]:
def bert_BiLSTM_CRF_model():
    
    ner_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_ner)
    bert_output = ner_model.layers[-9].output
    X = Lambda(lambda x: x[:, 0: input_shape[0]])(bert_output)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    output = CRF(3, sparse_target = True)(X)    
    ner_model = Model(ner_model.input, output)
    
    for layer in ner_model.layers:
        layer.trainable = False
    ner_model.layers[-1].trainable = True
    ner_model.layers[-2].trainable = True
    
    return ner_model

ner_model = bert_BiLSTM_CRF_model()

In [38]:
total_steps, warmup_steps = calc_train_steps(
    num_example=data.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [39]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_crf_accuracy', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [40]:
ner_model.compile(optimizer=optimizer,
                  loss=crf_loss,
                  metrics=[crf_accuracy])

ner_model.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size)
          #validation_split=0.1,
          #callbacks=callback_list)

Epoch 1/3
9256/9256 [==============================] - 1100s 119ms/step - loss: 0.0201 - crf_accuracy: 0.9958
Epoch 2/3
9256/9256 [==============================] - 1088s 118ms/step - loss: 0.0080 - crf_accuracy: 0.9978
Epoch 3/3
9256/9256 [==============================] - 1087s 117ms/step - loss: 0.0077 - crf_accuracy: 0.9979


# 4. 建立人名 AML 模型

In [41]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data = data[data['aml_label'] == 1]

data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data['name'] = data['name'].apply(lambda x: eval(x))
data['content'] = data['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data['content'] = data['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data['content'] = data['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
data['content'] = data['content'].apply(lambda x: re.sub('「.{1,4}」', '', x))
data['content'] = data['content'].apply(lambda x: re.sub('｜.', '。', x))

train = data

In [42]:
train = train[train['aml_label'] == 1]
train['news_id'] = range(1, len(train)+1)

In [43]:
maxlen_sentences = 256
batch_size = 8
epochs = 3

In [44]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content_model2(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)
    
    return data_more_split

In [45]:
def predict_sentences_model2(test, people_list, stick, tokenizer=tokenizer, maxlen=maxlen_sentences):
    
    AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
    aml_highrisk = np.asarray(test['content'])
    news_ids = np.asarray(test['news_id'])
    people_list = list(test['people_list'])
    
    for k, (news_id, y_news) in enumerate(zip(news_ids ,aml_highrisk)): 
        #用，。？切分句子
        y_news = y_news.replace('。','=。')
        y_news = y_news.replace('，','*，')
        y_news = y_news.replace('？','+？')
        y_news = y_news.replace('；','{；')
        
        news = re.split('，|。|？|；', y_news)
        
        news = [news.replace('=','。') for news in news]
        news = [news.replace('*','，') for news in news]
        news = [news.replace('+','？') for news in news]
        news = [news.replace('{','；') for news in news]
#############################################################################################################
        for i in range(len(people_list[k])):
            # 找出人名存在的 news index
            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                                
            x = [people for people in people_list[k] if people_list[k][i] not in people]

            name = [name for name in people_list[k] if name != people_list[k][i]]
            name = [name for name in name if ((len(name)<3) & (name[0] != people_list[k][i][0])) | (len(name)>=3)]
            name.sort(reverse=True)

            for j in index:

                mid = news[j]
                
                if name != []:
                    name = [name.replace('?', '\?') for name in name]
                    mid = re.sub('|'.join(name), '', mid)
                
                if j == 0:
                    if len(news) != 1:
                        end = news[j+1]  

                        if True in [people in news[j+1] for people in x]:
                            sentences = mid
                        else:
                            sentences = mid + end
                    elif len(news) == 1:
                        sentences = mid

                elif j+1 == len(news):
                    start = news[j-1]
                    if '。' in start:
                        start = ''

                    if True in [people in news[j-1] for people in x]:
                        sentences = mid
                    else:
                        sentences = start + mid

                else:
                    end = news[j+1]
                    start = news[j-1]                    

                    if '。' in start:
                        start = ''
                    
                    if '。' in mid:
                        end = ''                 

                    if (True not in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = start + mid + end
                    elif (True not in [people in news[j-1] for people in x]) & (True in [people in news[j+1] for people in x]):
                        sentences = start + mid
                    elif (True in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = mid + end
                    else:
                        sentences = mid  
   
                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
                #break  
    
    
    # 把指向同一人的姓名改成一樣（陳男 -> 陳水扁），若指向多人則不改（陳男 -> 陳致中、陳水扁）
    # 將預測不完整的名字回填（王音 -> 王音之）
    name_list = []
    for ids in AML['news_id'].unique():
        
        full_name = [name for name in AML[(AML['news_id'] == ids)]['Name']]
        full_3name = list(set([name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) in [3, 4]]))
        full_longname = list(set([name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) > 3]))
        
        a = Counter([name[0] for name in full_3name])
        keep = [k for k,v in a.items() if v == 1]

        name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # ex: {'王音': '王音之'}
        name_dict_3 = dict(zip([name[1:] for name in full_longname], full_longname))
        name_dict_4 = dict(zip([name[:2] for name in full_longname], full_longname))
        
        for name in full_name:
            if (name in name_dict_2.keys()) & (len(name) == 2):
                name_list.append(name_dict_2.get(name))
            elif (name in name_dict_3.keys()):
                name_list.append(name_dict_3.get(name))
            elif (name in name_dict_4.keys()):
                name_list.append(name_dict_4.get(name))            
            else:
                name_list.append(name)
                
    
    # 排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
    AML['Name'] = name_list
    AML = AML.drop_duplicates()
    AML = AML[AML['Name'].apply(lambda x: (len(x) > 1) )]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                   
                                                                     '員', '稱', '家', '哥', '媽', '生', \
                                                                     '揆', '董', '母', '公', '少', '翁',\
                                                                     '粉', '仔', '氏', '父','童', '弟', '嬤','姊','姐','妹', '警']))]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[0] in ['老', '小', '阿']))]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] == x[0]))]
    return AML

In [46]:
# 取得名字 (預測結果為onehot的狀態)
def get_name(input_id, y_pred):
    
    label_list = []
    word_dict = {v: k for k, v in token_dict.items()}
    
    for input_data, y in zip(input_id, y_pred):
        people_index = ''.join([str(a) for a in list(y)])
        j = 0
        name_list = []
        split_index = re.findall('[12]2*', people_index)
        name = ''.join([word_dict.get(input_data[index]) for index, value in enumerate(y) if value != 0])
        
        # [UNK], [PAD]會被算成 5 個字元，避免轉換成文字的index因長度不同對不上，故用 1 個字元的其他符號替代
        # 王春甡 -> 王春[UNK] -> 王春?
        name = name.replace('[UNK]','?')
        name = name.replace('[PAD]','!')
        
        for i in split_index:
            name_list.append(name[0+j:len(i)+j])
            j = len(i) + j
            
        name_list = [name for name in name_list]
        label_list.append(list(set(name_list)))
    
    return label_list

In [47]:
# 2. 將超過 512 的句子以句點拆成多句分段預測
train_ner = train.drop(['name'], axis=1)
data_less = train_ner[train_ner['content'].str.len() <= 512]
data_more = train_ner[train_ner['content'].str.len() > 512]
data_more_split = split_content_model2(data_more)
train_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

# 3. NER 預測人名
input_id, segment_id, mask_input = encoded(tokenizer, train_ner, maxlen=maxlen_ner)
prediction = ner_model.predict([input_id, segment_id, mask_input])
y_pred = np.argmax(prediction, axis=-1)
people_list = get_name(input_id, y_pred)

In [48]:
# 2. 將超過 512 的句子以句點拆成多句分段預測
train_ner = train.drop(['name'], axis=1)
data_less = train_ner[train_ner['content'].str.len() <= 512]
data_more = train_ner[train_ner['content'].str.len() > 512]
data_more_split = split_content_model2(data_more)
train_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

# 3. NER 預測人名
input_id, segment_id, mask_input = encoded(tokenizer, train_ner, maxlen=maxlen_ner)
prediction = ner_model.predict([input_id, segment_id, mask_input])
y_pred = np.argmax(prediction, axis=-1)
people_list = get_name(input_id, y_pred)


# 4. 將拆開的句子組合回去
train_ner['people_list'] = people_list
content = train_ner[['news_id', 'content', 'aml_label']]
content = content.groupby(['news_id', 'aml_label'])['content'].apply(lambda x : '。'.join(x)).reset_index()
people = train_ner[['news_id', 'aml_label', 'people_list']]
people = people.groupby(['news_id', 'aml_label'])['people_list'].agg(sum).reset_index()
people['people_list'] = [list(set(people)) for people in people['people_list']]
train_ner = pd.merge(content, people, on=['news_id', 'aml_label'], how='left')

# 5. 將 [UNK], [PAD] 轉換回來 (王春? -> 王春甡)
for _, row in train_ner.iterrows():
    for i, name in enumerate(row['people_list']):
        if ('?' in name) | ('!' in name):
            reexp = name.replace('?', '.').replace('!', '.')
            reexp = re.compile(reexp,re.IGNORECASE)
            row['people_list'][i] = re.search(reexp, row['content']).group()
            
AML = predict_sentences_model2(train_ner, list(train_ner['people_list']), tokenizer=tokenizer, maxlen=maxlen_sentences, stick=False)

#合併句子
AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()

train_name = train.drop(['content'], axis=1)
s = train_name.apply(lambda x: pd.Series(x['name']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Name'
train_name = train_name.drop('name', axis=1).join(s)

aml_train = pd.merge(AML, train_name, on=['news_id', 'Name'], how='left')
aml_train = aml_train.fillna(0)

In [49]:
len(aml_train)

1058

In [50]:
len(aml_train[aml_train['aml_label'] == 1])

796

In [51]:
label = np.asarray(aml_train['aml_label'])
input_id, segment_id, mask_input = encoded(tokenizer, aml_train, maxlen_sentences)

In [52]:
def bert_model():
    
    model2 = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_sentences)
    sequence_output = model2.layers[-6].output
    sequence_output = Dense(16, activation='relu')(sequence_output)
    sequence_output = Dense(16, activation='relu')(sequence_output)
    output = Dense(1, activation='sigmoid')(sequence_output)
    model2 = Model(model2.input, output)

    return model2

model2 = bert_model()

set_trainable = False
for layer in model2.layers:
    if layer.name == 'Encoder-12-MultiHeadSelfAttention':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [53]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [54]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [55]:
model2.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model2.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )

Epoch 1/3
1058/1058 [==============================] - 17s 16ms/step - loss: 0.4349 - acc: 0.8129
Epoch 2/3
1058/1058 [==============================] - 15s 14ms/step - loss: 0.2830 - acc: 0.8904
Epoch 3/3
1058/1058 [==============================] - 14s 14ms/step - loss: 0.2711 - acc: 0.9008


In [56]:
model.save_weights('model1_0801官方.h5')
model_aml.save_weights('model1.5_0801官方.h5')
model2.save_weights('model2_0801官方.h5')